# Phase 1


## Load Libraries

In [1]:
# !pip install geopandas
# import geopandas as gpd

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import math
import re
import datetime
from shapely import wkt
import urllib
import json
import csv
import requests
import jdc
import ipynb

## APIs for pulling in permit files

In [30]:
austin = pd.DataFrame()
chicago = pd.DataFrame()
sanfran = pd.DataFrame()
nyc = pd.DataFrame()

urls = [
    'https://data.austintexas.gov/resource/3syk-w9eu.csv?$limit=5000&$order=applieddate%20DESC&$where=' + urllib.parse.quote('applieddate IS NOT NULL'),
    'https://data.cityofchicago.org/resource/ydr8-5enu.csv?$limit=5000&$order=application_start_date%20DESC&$where=' + urllib.parse.quote('application_start_date IS NOT NULL'),
    'https://data.sfgov.org/resource/i98e-djp9.csv?$limit=500&$order=permit_creation_date%20DESC&$where=' + urllib.parse.quote('permit_creation_date IS NOT NULL'),
    'https://data.cityofnewyork.us/resource/ipu4-2q9a.csv?$limit=500&$order=filing_date%20DESC&$where=' + urllib.parse.quote('filing_date IS NOT NULL'),
]

for url in urls:
  if re.search('data.austintexas.gov', url):
    austin = pd.read_csv(url)
    austin.name = 'Austin'
  elif re.search('data.cityofchicago.org', url):
    chicago = pd.read_csv(url)
    chicago.name = 'Chicago'
  elif re.search('data.sfgov.org', url):
    sanfran = pd.read_csv(url)
    sanfran.name = 'San Francisco'
  elif re.search('data.cityofnewyork.us', url):
    nyc = pd.read_csv(url)
    nyc.name = 'New York City'

## Initialize Data Spec

Upload fields.csv and desired permits file for any city

In [7]:
#eventually, the idea will be for permits to change based on the API call and then a city specific dataframe will be created.
#Once all API calls are made, the city specific dataframes will all be concatenated and exported
permits = sanfran

In [50]:
#Initializing the data frame for our final data specification
fields = pd.read_csv('fields.csv', usecols = ['Combined'])
ls_fields = list(fields.Combined.unique())
dataspec = pd.DataFrame(columns = ls_fields, index = [])

In [25]:
#import schema functions
from ipynb.fs.full.schema_functions import geometry

In [26]:
def functions(city):
    geometry(city)
    return city

In [31]:
cities = [austin, chicago, sanfran, nyc]

for c in cities:
    print(c.name)
    geometry(c)

Austin
Chicago


ParseException: Unknown type: 'ROOF'


WKTReadingError: Could not create geometry because of errors while reading input.

In [38]:

chicago.iloc[:1,110:]

,pin10,community_area,census_tract,ward,xcoordinate,ycoordinate,latitude,longitude,location
0,nan,15.0,151100.0,30.0,1139376.66557001,1921444.2054039924,41.940556,-87.763153,POINT (-87.763153195317 41.940555558327)


In [13]:
permits.head(1)

,permit_number,permit_type,permit_type_definition,permit_creation_date,block,lot,street_number,street_number_suffix,street_name,street_suffix,...,adu,primary_address_flag,supervisor_district,neighborhoods_analysis_boundaries,zipcode,location,point_source,record_id,data_as_of,data_loaded_at
0,202306260894,8,otc alterations permit,2023-06-26T15:47:02.000,1430,022,210,nan,Clement,St,...,N,Y,1,Inner Richmond,94118.0,POINT (-122.461497167 37.783329936),parcel_centroid,1672039358767,2023-06-26T23:55:00.000,2023-06-27T04:12:11.000


## Creating Geo Data Frames

If the dataframe already has a point geometry, this will create latitude and longitude columns

In [ ]:
geom_count = 0

for col in permits.columns:
  permits[col] = permits[col].astype(str)
  if permits[col].str.contains('POINT').any():
    geom_count += 1
    geo = col
  else:
    continue

if geom_count > 1:
  raise ValueError('More than one POINT geometry column. Investigate.')
elif geom_count == 1:
  permits = permits[permits[geo].notna()]
  permits['geometry'] = permits[geo]

  permits['geometry'] = permits['geometry'].replace('nan',None)
  permits['geometry'] = permits['geometry'].apply(wkt.loads)

  permits = gpd.GeoDataFrame(permits, geometry = permits['geometry'], crs=4326)
  permits['Longitude'] = permits['geometry'].x
  permits['Latitude'] = permits['geometry'].y
else:
  permits = permits

ERROR


# Phase 2.1: Schema Fields 1 - 13

## Unique Land IDs and Unique Building IDs

In [ ]:
#Checking for Land IDs
for col in permits.columns:
  # if pd.Series(dataspec['Unique Land Identifier'].values).isnull().all() == True:
  if len(dataspec['Unique Land Identifier'].value_counts()) == 0:
    if re.search(r'(^pin|parcel|pin$)', col, re.IGNORECASE):
      dataspec['Unique Land Identifier'] = pd.Series(permits[col].values)
      break
    elif re.search('folio', col, re.IGNORECASE):
      dataspec['Unique Land Identifier'] = pd.Series(permits[col].values)
      break
    else: continue
  else: continue

In [ ]:
#Checking for Land IDs or Building IDs
for col in permits.columns:
  if re.search(r'(^id\b|[^a-zA-Z]id[^a-zA-Z]|\bid$)', col, re.IGNORECASE):
    if len(dataspec[r'Unique Land Identifier'].value_counts()) == 0:
      if len(dataspec[r'Unique Building Identifier'].value_counts()) == 0:
        if re.search(r'build|bin|property', col, re.IGNORECASE):
          dataspec['Unique Building Identifier'] = pd.Series(permits[col].values)
          break
        else:
          dataspec['Unique Land Identifier'] = pd.Series(permits[col].values)
          break
      else: continue
    else: continue
  else: continue

## Address, Street Number, and Street Name



### Street Number

In [ ]:
st_num1 = []
st_num2 = []

numsufcount = 0

for col in permits.columns:
  if re.search(r'(address|street|st\.?\b)', col, re.IGNORECASE):
    if not re.search(r'(cont|appli|2|flag|owner)', col, re.IGNORECASE):
      if re.search(r'(no\.?\b|num|number|#|house)', col, re.IGNORECASE):
        if re.search(r'(suffix|suf)', col, re.IGNORECASE):
          numsufcount +=1

for col in permits.columns:
  if re.search(r'(address|street|st)', col, re.IGNORECASE):
    if not re.search(r'cont|appli|2|flag', col, re.IGNORECASE):
      if re.search(r'(no\.?\b|num|number|#|house)', col, re.IGNORECASE):
        if numsufcount != 0:
          if re.search(r'(suffix|suf)', col, re.IGNORECASE):
            st_num2 = pd.Series(permits[col].values).fillna('').astype(str)
        else: continue

In [ ]:
num_full = pd.Series()

for col in permits.columns:
  if re.search(r'(address|street|st\.?\b)', col, re.IGNORECASE):
    if not re.search(r'(cont|appli|2|flag|owner)', col, re.IGNORECASE):
      if re.search(r'(no\.?\b|num|number|#|house)', col, re.IGNORECASE):
        if not re.search(r'(suffix|suf)', col, re.IGNORECASE):
          if numsufcount != 0:
            st_num1 = pd.Series(permits[col].values).astype(str)
            num_full = pd.Series(st_num1.str.cat(st_num2, sep=""))
            dataspec['Building Street Number'] = num_full
            break
          elif numsufcount == 0:
            st_num1 = pd.Series(permits[col].values).astype(str)
            num_full = st_num1
            dataspec['Building Street Number'] = num_full
            break

<ipython-input-179-acbba0bfe146>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  num_full = pd.Series()


### Street Name/Address

In [ ]:
#For When there's Direction and Suffix in a Street Name (Which are different columns than the Street's NAME)
st_name1 = [] #bookend 1
st_name2 = [] #The Actual Street Name
st_name3 = [] #bookend 2

dircount = 0 #checking for the existence of 'Direction' in the list of columns: N, E, S, W, North, East, etc.
sufcount = 0 #checking for the existence of 'Suffix' in the list of columns: Ave, Blvd, St, Rd, Street, Avenue, etc.
for col in permits.columns:
  if re.search(r'(address|street|\bst\.?\b)', col, re.IGNORECASE):
    if not re.search(r'cont|appli|2|flag', col, re.IGNORECASE):
      if not re.search(r'(no\.?|num|number|#|house)', col, re.IGNORECASE): #neither direction or suffix counts here have anything to do with street number
        if re.search(r'direction', col, re.IGNORECASE): #may be other variables
          dircount+=1
        elif re.search(r'(suffix|suf)', col, re.IGNORECASE): #may be other variables
          sufcount+=1
  elif col.lower() == "suffix":
    sufcount+=1

for col in permits.columns:
  if re.search(r'(address|street|\bst\.?\b)', col, re.IGNORECASE):
    if not re.search(r'cont|appli|2|flag', col, re.IGNORECASE):
      if not re.search(r'(no\.?|num|number|#|house)', col, re.IGNORECASE):
        if dircount != 0:
          if re.search(r'direction', col, re.IGNORECASE):
            st_name1 = pd.Series(permits[col].values)
          else: continue
        elif sufcount != 0:
          if re.search(r'(suffix|suf)', col, re.IGNORECASE):
            st_name3 = pd.Series(permits[col].values)
          else: continue
  elif col.lower() == "suffix":
    if sufcount != 0: #suffix by itself - most likely street name suffix, not street number suffix
      if re.search(r'(suffix|suf)', col, re.IGNORECASE):
        st_name3 = pd.Series(permits[col].values)
      else: continue

In [ ]:
#Concatenating Street Name Related Columns
st_full = pd.Series()

for col in permits.columns:
  if re.search(r'(address|street|\bst\.?\b)', col, re.IGNORECASE):
    if not re.search('cont|appli|2|flag', col, re.IGNORECASE):
      if re.search(r'name', col, re.IGNORECASE):
        if (dircount != 0) & (sufcount != 0):
          st_name2 = pd.Series(permits[col].values)
          st_full = pd.Series(st_name1.str.cat([st_name2, st_name3], sep=" "))
          dataspec['Building Street Name'] = st_full
          break
        elif (dircount != 0) & (sufcount == 0):
          st_name2 = pd.Series(permits[col].values)
          st_full = pd.Series(st_name1.str.cat(st_name2, sep=" "))
          dataspec['Building Street Name'] = st_full
          break
        elif (dircount == 0) & (sufcount != 0):
          st_name2 = pd.Series(permits[col].values)
          st_full = pd.Series(st_name2.str.cat(st_name3, sep=" "))
          dataspec['Building Street Name'] = st_full
          break
        elif (len(dataspec['Building Street Number'].value_counts()) != 0) & ((dircount == 0) & (sufcount == 0)):
          st_name2 = pd.Series(permits[col].values)
          st_full = st_name2
          dataspec['Building Street Name'] = st_full
          break
      elif (len(dataspec['Building Street Number'].value_counts()) == 0) & ((dircount == 0) & (sufcount == 0)):
        dataspec[['Building Street Number', 'Building Street Name']] = pd.Series(permits[col].values).str.split(n=1,expand=True)
        break
      else: continue
    else: continue
  else: continue

<ipython-input-181-a4df76b122e3>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  st_full = pd.Series()


## City, State, and Zip Code

Some cities may not have this information, despite other identifying geographic factors such as latitude/longitude.

In [ ]:
for col in permits.columns:
  if re.search('city|boro',col,re.IGNORECASE):
    if not re.search('cont|appli|owner', col, re.IGNORECASE):
      dataspec['Building City'] = pd.Series(permits[col].values)

  if re.search('state', col, re.IGNORECASE):
    if not re.search('id|house|_|lic|con', col, re.IGNORECASE):
      dataspec['Building State'] = pd.Series(permits[col].values)

  if re.search('zip', col, re.IGNORECASE):
    if not re.search('cont|appli|house|2', col, re.IGNORECASE):
      dataspec['Building Zip Code'] = pd.Series(permits[col].values)
      dataspec['Building Zip Code'] = dataspec['Building Zip Code'].fillna(0)

  if re.search('country',col,re.IGNORECASE):
    if not re.search('cont|appli|owner', col, re.IGNORECASE):
      dataspec['Building country'] = pd.Series(permits[col].values)

## Building Type/Land-Use (Existing and Proposed)

In [ ]:
#Existing Land Use/Building Type - Unfortunately not easily separable
for col in permits.columns:
  if re.search(r'typ|map|use', col, re.IGNORECASE):
    if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE): #if the column contains existing|exist|not 'proposed'
      if re.search(r'building|bldg|land', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break
        elif re.search(r'typ', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break
      elif re.search(r'use', col, re.IGNORECASE):
        dataspec['Existing Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'typ', col, re.IGNORECASE):
        dataspec['Existing Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'map', col, re.IGNORECASE):
        dataspec['Existing Land Use'] = pd.Series(permits[col].values)
        break


for col in permits.columns:
  if len(dataspec['Existing Land Use'].value_counts()) != 0:
    break
  else:
    if re.search(r'typ|map|use', col, re.IGNORECASE):
      if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE):
        if re.search(r'occupancy', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break
      elif re.search(r'occupancy', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break

In [ ]:
#Proposed Land Use/Building Type - Unfortunately not easily separable
for col in permits.columns:
  if re.search(r'typ|map|use', col, re.IGNORECASE):
    if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
      print(col)
      if re.search(r'building|bldg|land', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
          break
        elif re.search(r'typ', col, re.IGNORECASE):
          dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
          break
      elif re.search(r'use', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'typ', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'map', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
    elif len(dataspec['Proposed Land Use'].value_counts()) == 0:
      if re.search(r'use', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'map', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break

for col in permits.columns:
  if len(dataspec['Proposed Land Use'].value_counts()) != 0:
    break
  else:
    if re.search(r'typ|map|use', col, re.IGNORECASE):
      if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
        if re.search(r'occupancy', col, re.IGNORECASE):
          dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
          break

## Zoning Designation (Existing and Proposed)

In [ ]:
#Existing and Proposed Zoning Designations
for col in permits.columns:
  if re.search(r'zoning|zon', col, re.IGNORECASE):
    if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE):
      if re.search(r'use|typ', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Existing Zoning Designation'] = pd.Series(permits[col].values)
          break
        if re.search(r'typ', col, re.IGNORECASE):
          dataspec['Existing Zoning Designation'] = pd.Series(permits[col].values)
          break

for col in permits.columns:
  if re.search(r'zoning|zon', col, re.IGNORECASE):
    if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
      if re.search(r'use|typ', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Proposed Zoning Designation'] = pd.Series(permits[col].values)
          break
        if re.search(r'typ', col, re.IGNORECASE):
          dataspec['Proposed Zoning Designation'] = pd.Series(permits[col].values)
          break

# Phase 2.1: Testing

In [ ]:
dataspec.iloc[:5,:13]

,Unique Building Identifier,Unique Land Identifier,Building Street Number,Building Street Name,Building City,Building State,Building Zip Code,Building Country,Building Type,Existing Land Use,Proposed Land Use,Existing Zoning Designation,Proposed Zoning Designation
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commercial,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commercial,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Residential,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Residential,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Residential,NaN,NaN


# Phase 2.2: Schema Fields 14 - 26

## Permit Number

In [ ]:
for col in permits.columns:
  if re.search('permit', col, re.IGNORECASE):
    if re.search(r'(no\.?\b|num|number|#)', col, re.IGNORECASE):
      if not re.search('master', col, re.IGNORECASE):
        dataspec['Permit Number'] = pd.Series(permits[col].values)
      else: continue
    else: continue
  else: continue

## Permit Type/Sub-Type, Work Type/Sub-Type, Work Description

In [ ]:
for col in permits.columns:
  if re.search(r'permit', col, re.IGNORECASE):
    if re.search(r'typ|class', col, re.IGNORECASE): #searching for permit type
      if not re.search(r'definit|descr|work', col, re.IGNORECASE): #will relegate definition/description elsewhere
        if not re.search(r'sub', col, re.IGNORECASE):
          dataspec['Permit Type'] = pd.Series(permits[col].values)
        else:
          dataspec['Permit Sub-Type'] = pd.Series(permits[col].values)
      else:
        if not re.search(r'work', col, re.IGNORECASE):
          if len(dataspec['Permit Type'].value_counts()) == 0:
            dataspec['Permit Type'] = pd.Series(permits[col].values)
          elif len(dataspec['Permit Sub-Type'].value_counts()) == 0:
            dataspec['Permit Sub-Type'] = pd.Series(permits[col].values)
          else: continue
        else: continue
    else: continue

In [ ]:
for col in permits.columns:
  if re.search(r'work|action', col, re.IGNORECASE):
    if re.search(r'typ', col, re.IGNORECASE): #searching for permit type
      if not re.search(r'definit|descr', col, re.IGNORECASE):
        if not re.search(r'sub', col, re.IGNORECASE):
          dataspec['Work Type'] = pd.Series(permits[col].values)
        else:
          dataspec['Work Sub-Type'] = pd.Series(permits[col].values)
      else:
        if len(dataspec['Work Type'].value_counts()) == 0:
          dataspec['Work Type'] = pd.Series(permits[col].values)
        elif len(dataspec['Work Sub-Type'].value_counts()) == 0:
          dataspec['Work Sub-Type'] = pd.Series(permits[col].values)
    else: continue
  else: continue

In [ ]:
for col in permits.columns:
  if re.search(r'definit|descr', col, re.IGNORECASE):
    if not re.search(r'permit|work', col, re.IGNORECASE):
      dataspec['Project Description'] = pd.Series(permits[col].values)
      break #finds the first column that mentions description without 'permit' or 'work' --> might need more rules
    else: continue
  else: continue

## Dates: Permits, Permit Status, Certificate of Occupancy

Completed Date may mean very different things depending on the city. It is assumed NOT to mean: Permit Expiration Date or Certificate of Occupancy Issued Date.

For our purposes, we will assume Completed Date means Inspection Completion Date

In [ ]:
for col in permits.columns:
  if re.search(r'date', col, re.IGNORECASE): #focus first on 'permits' related dates, then other dates
    if re.search(r'appli|creat', col, re.IGNORECASE): #capturing Application and Creation Date fields
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Application Date'] = pd.Series(permits[col].values)
    elif re.search(r'approv', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Approval Date'] = pd.Series(permits[col].values)
    elif re.search(r'issu', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Issued Date'] = pd.Series(permits[col].values)
    elif re.search(r'expir', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Expiration Date'] = pd.Series(permits[col].values)
    elif re.search(r'comple', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Inspection Completion Date'] = pd.Series(permits[col].values)
    else: #check for other variations, but avoid connecting 'certificate of occupancy', 'construction', 'status' dates, etc.
      continue

if (len(dataspec['Permit Approval Date'].value_counts()) == 0) & (len(dataspec['Permit Issued Date'].value_counts()) != 0):
        dataspec['Permit Approval Date'] = dataspec['Permit Issued Date']

AppliedDate


In [ ]:
for col in permits.columns:
  if re.search(r'status', col, re.IGNORECASE):
    if re.search(r'date', col, re.IGNORECASE):
      dataspec['Permit Current Status Date'] = pd.Series(permits[col].values)
    else:
      dataspec['Permit Current Status'] = pd.Series(permits[col].values)
  else: continue

In [ ]:
for col in permits.columns:
  if re.search(r'date', col, re.IGNORECASE):
    if re.search(r'cert|occup', col, re.IGNORECASE):
      dataspec['Certificate of Occupancy Issued Date'] = pd.Series(permits[col].values)

# Phase 2.2: Testing

In [ ]:
dataspec.iloc[:5,13:26]

,Permit Number,Permit Application Date,Permit Approval Date,Permit Issued Date,Permit Expiration Date,Certificate of Occupancy Issued Date,Permit Type,Permit Sub-Type,Work Type,Work Sub-Type,Project Description,Permit Current Status,Permit Current Status Date
0,BLD-ACC2023-00048,2023-06-15,NaT,NaT,NaN,NaN,Building Permit - Accessory,NaN,Alteration,NaN,Replace the existing garage roof and exterior ...,Received,NaN
1,BLD-MFD2023-00046,2023-06-15,NaT,NaT,NaN,NaN,Building Permit - Multifamily,NaN,New,NaN,Test permit case used by system administrators.,Received,NaN
2,BLD-NRE2023-00098,2023-06-15,NaT,NaT,NaN,NaN,Building Permit - Non-Residential,NaN,Tenant Finish,NaN,Interior Tenant Renovation,Received,NaN
3,BLD-NRE2023-00099,2023-06-15,NaT,NaT,NaN,NaN,Building Permit - Non-Residential,NaN,Addition and Alteration,NaN,AT BUILDING 6036:\n-TWO NEW EXTERIOR CANOPIES ...,Received,NaN
4,ELE2023-00608,2023-06-15,NaT,NaT,NaN,NaN,Electrical Permit,NaN,Electrical Permit,NaN,Install 240V 60A hardwired Tesla wall connecto...,Received - Online,NaN


# Phase 2.3: Schema Fields 28 - 40

## Floors, Floor/Roof Rail Height, Building Height

Assumptions: Floor Height = 10 feet, Rooftop Rail Height = 4 ft

In [ ]:
for col in permits.columns:
  if re.search(r'floor|\bstor', col, re.IGNORECASE):
    if re.search(r'(no\.?\b|num|number|#)', col, re.IGNORECASE):
      if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE):
        permits[col] = permits[col].fillna(0)
        dataspec['Existing Number of Floors Above Grade'] = pd.Series(permits[col].values).replace('nan',0).astype(float).astype(int)
        break
      else: continue
    else: continue
  else: continue

for col in permits.columns:
  if re.search(r'floor|\bstor', col, re.IGNORECASE):
    if re.search(r'(no\.?\b|num|number|#)', col, re.IGNORECASE):
      if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
        dataspec['Proposed Number of Floors Above Grade'] = pd.Series(permits[col].values).replace('nan',0).astype(float).astype(int)
        break
      else:
        if len(dataspec['Existing Number of Floors Above Grade'].value_counts()) == 0:
          dataspec['Existing Number of Floors Above Grade'] = pd.Series(permits[col].values).replace('nan',0).astype(float).astype(int)
          break
        else: continue
    else: continue
  else: continue

In [ ]:
dataspec.loc[dataspec['Existing Number of Floors Above Grade'].notnull(), 'Existing Building Height'] = dataspec['Existing Number of Floors Above Grade']*10+4
dataspec.loc[dataspec['Proposed Number of Floors Above Grade'].notnull(), 'Proposed Building Height'] = (dataspec['Proposed Number of Floors Above Grade']*10)+4

## Geometries: Latitude, Longitude, Footprint (as applicable)

In [ ]:
for col in permits.columns:
  if re.search(r'latitude|point', col, re.IGNORECASE):
    if re.search(r'latitude|y', col, re.IGNORECASE):
      dataspec['Latitude'] = pd.Series(permits[col].values)

for col in permits.columns:
  if re.search(r'longitude|point', col, re.IGNORECASE):
    if re.search(r'longitude|x', col, re.IGNORECASE):
      dataspec['Longitude'] = pd.Series(permits[col].values)

# Phase 2.3: Testing

In [ ]:
dataspec.iloc[:5,26:38]

,Existing Number of Floors Above Grade,Proposed Number of Floors Above Grade,Number of Floors Below Grade,Floor Height,Existing Building Height,Proposed Building Height,Gross Floor Area,Inspection Type,Inspection Completion Date,Latitude,Longitude,Building Footprint
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


# Archive

In [ ]:
for col in permits.columns:



    if re.search('construction|bldg|commercial',col,re.IGNORECASE):
        if not re.search('sqft|date|exist|pro',col,re.IGNORECASE):
            dataspec['Building Type'] = pd.Series(permits[col].values)

    if re.search('occupancy|class',col,re.IGNORECASE):
        if re.search('type',col,re.IGNORECASE):
            dataspec['Building Type'] = pd.Series(permits[col].values)

    if re.search('class',col,re.IGNORECASE):
        if re.search('mapped',col,re.IGNORECASE):
             dataspec['Building Type'] = pd.Series(permits[col].values)

    if re.search('group|proposed',col,re.IGNORECASE):
        if not re.search('code|des|stor|unit|use|con',col,re.IGNORECASE):
            dataspec['Land-Use'] = pd.Series(permits[col].values)

    if re.search('occupancy',col,re.IGNORECASE):
        if re.search('type',col,re.IGNORECASE):
            dataspec['Land-Use'] = pd.Series(permits[col].values)

###########################################################################

    if re.search('num', col, re.IGNORECASE):
        if re.search('permit|ivr', col, re.IGNORECASE):
            if not re.search('master', col, re.IGNORECASE):
                dataspec['Permit Number'] = pd.Series(permits[col].values)

    if col == 'PERMIT#':
        dataspec['Permit Number'] = pd.Series(permits[col].values)

    if col == 'Job #':
        dataspec['Permit Number'] = pd.Series(permits[col].values)

    if re.search('date', col, re.IGNORECASE):
        if re.search('appli|start|creation', col, re.IGNORECASE):
            dataspec['Permit Application Date'] = pd.Series(permits[col].values)

    if re.search('set', col, re.IGNORECASE):
        if not re.search('photovo|plan', col, re.IGNORECASE):
            dataspec['Permit Application Date'] = pd.Series(permits[col].values)

    if re.search('issu', col, re.IGNORECASE):
        if re.search('date', col, re.IGNORECASE):
            if not re.search('coi', col, re.IGNORECASE):
                dataspec['Permit Issued Date'] = pd.Series(permits[col].values)

    if col == 'issued':
        dataspec['Permit Issued Date'] = pd.Series(permits[col].values)

    if re.search('date', col, re.IGNORECASE):
        if re.search('completed|expir', col, re.IGNORECASE):
            if not re.search('valuation', col, re.IGNORECASE):
                dataspec['Permit Expiration Date'] = pd.Series(permits[col].values)

    if re.search('final', col, re.IGNORECASE):
        if not re.search('valuation', col, re.IGNORECASE):
            dataspec['Permit Expiration Date'] = pd.Series(permits[col].values)
###########################################################################

    if re.search('type', col, re.IGNORECASE):
        if re.search('permit', col, re.IGNORECASE):
            if not re.search('work|sub|lic|desc|map|def', col, re.IGNORECASE):
                dataspec['Permit Type'] = pd.Series(permits[col].values)

    if col == 'type':
        dataspec['Permit Type'] = pd.Series(permits[col].values)

    if re.search('type', col, re.IGNORECASE):
        if re.search('sub|work|action', col, re.IGNORECASE):
            if not re.search('permit', col, re.IGNORECASE):
                dataspec['Work Type'] = pd.Series(permits[col].values)

    if re.search('type', col, re.IGNORECASE):
        if re.search('permit', col, re.IGNORECASE):
            if re.search('def', col, re.IGNORECASE):
                dataspec['Work Type'] = pd.Series(permits[col].values)

    if col == 'work':
        dataspec['Work Type'] = pd.Series(permits[col].values)

    if re.search('class|scope', col, re.IGNORECASE):
        if re.search('work', col, re.IGNORECASE):
            if not re.search('map', col, re.IGNORECASE):
                dataspec['Work Type'] = pd.Series(permits[col].values)

    if re.search('status', col, re.IGNORECASE):
        if not re.search('filing|date', col, re.IGNORECASE):
            dataspec['Permit Current Status'] = pd.Series(permits[col].values)

    if re.search('status', col, re.IGNORECASE):
        if re.search('date', col, re.IGNORECASE):
            dataspec['Permit Current Status Date'] = pd.Series(permits[col].values)

    if re.search('number', col, re.IGNORECASE):
        if re.search('floor|stories', col, re.IGNORECASE):
            if not re.search('pro', col, re.IGNORECASE):
                dataspec['Number of Floors Above Grade'] = pd.Series(permits[col].values)

    if re.search('y_web|lat|pointy', col, re.IGNORECASE):
        if not re.search('soc|mup', col, re.IGNORECASE):
            dataspec['Latitude'] = pd.Series(permits[col].values)

    if re.search('x_web|long|pointx', col, re.IGNORECASE):
        if not re.search('soc', col, re.IGNORECASE):
            dataspec['Longitude'] = pd.Series(permits[col].values)

#####################################################################
